# Modelo de Optimización:

__Problema:__ Cómo distribuir geográficamente patrullas a partir de una predicción de crimenes para una fecha dada y en una zona específica.

Se plantea una solución de Optimización Lineal basada en [Wheeler(2018).](https://www.emerald.com/insight/content/doi/10.1108/PIJPSM-02-2018-0027/full/html?skipTracking=true)

Para entender el modelo usamos el siguiente ejemplo, supongamos que tenemos la siguiente predicción:

![title](https://raw.githubusercontent.com/jscanass/modeling/master/ej1.png)

La solución que se plante es __dividir el mapa__ según las patrullas disponibles y para cada sub-mapa que se genera, asginar una zona como el punto más caliente(base). En este caso lo que se quiere minimizar es el __viaje ponderado total__ entre las áreas según el peso que cada una tenga y las distancias entre las áreas.

Para nuestro ejemplo suponemos 2 patrullas para asignar. ¿Cómo queda dividido el mapa?

In [1]:
#Ejemplo de Entradas de la función

Areas = ['A','B','C','D']
Calls = {'A': 4,
'B': 1,
'C': 2,
'D': 3}
Distances = {'A': {'A': 0, 'B': 1, 'C': 2, 'D': 3},
'B': {'A': 1, 'B': 0, 'C': 1, 'D': 2},
'C': {'A': 2, 'B': 1, 'C': 0, 'D': 1},
'D': {'A': 3, 'B': 2, 'C': 1, 'D': 0}}
Contiguity = {'A': ['B'],
'B': ['A','C'],
'C': ['B','D'],
'D': ['C']}

In [2]:
from pulp import * 
import numpy as np
from scipy.spatial import distance

In [3]:
TotalBeats = 2 #Sets the number of areas to cluster
#This says what the maximum amount of allowable calls in a particular area
SumCalls = sum(Calls.values())
Disparity = 1.2
MaxIneq = (SumCalls/TotalBeats)*(1 + Disparity) #20% disparity allowed

In [4]:
#1 set up the problem
ChoosePatrol = LpProblem("Creating Patrol Areas",LpMinimize)
#2 create decision variables
assign_areas = LpVariable.dicts("Sources and Destinations", [(s,d) for s in Areas for d in Areas], lowBound=0, upBound=1, cat=LpInteger)
#3 set up the function to minimize
ChoosePatrol += lpSum(Distances[s][d]*Calls[d]*assign_areas[(s,d)] for s in Areas for d in Areas), "Minimize weighted travel"
#4 Constraint - maximum number of beats
ChoosePatrol += lpSum(assign_areas[(s,s)] for s in Areas) == TotalBeats, "Setting Max number of beats"
#5 Constraint - No offbeat if local beat is not assigned
for s in Areas:
    for d in Areas:
        ChoosePatrol += assign_areas[s,d] <= assign_areas[s,s], "No off-beat if local beat is not assigned, %s %s" % (s,d)
#6 Constraint - every destination area must be covered at least once
for d in Areas:
    ChoosePatrol += sum(assign_areas[(s,d)] for s in Areas) == 1, "Destination Area %s must be covered at least once" % (d)
#7 Constraint - no source area should have too high a workload
for s in Areas:  
    ChoosePatrol += sum(assign_areas[(s,d)]*Calls[d] for d in Areas) <= MaxIneq, "Source Area %s needs to have less call weight than %s"% (s,MaxIneq)
#8 Constraint - areas need to be contiguous
for s in Areas:
    for d in Areas:
        ChoosePatrol += lpSum(assign_areas[(s,a)] for a in Contiguity[d]) >= assign_areas[(s,d)]


In [5]:
ChoosePatrol.solve()
print("Status is %s, The total weighted travel is %d" % (LpStatus[ChoosePatrol.status],value(ChoosePatrol.objective)))
results = []
for s in Areas:
    for d in Areas:
        if assign_areas[(s,d)].varValue == 1.0:
            results.append((s,d,Distances[s][d],Calls[d],Calls[d]*Distances[s][d]))

Status is Optimal, The total weighted travel is 3


In [6]:
# Resultado para cada tupla:
# 1: Origen
# 2: Destino
# 3: Distancia Origen-Destino
# 4: Peso del Destino
# 5: Viaje Origen-Destino

# El viaje ponderado total se calcula 
# sumando la quinta componente de cada tupla

results

[('A', 'A', 0, 4, 0),
 ('A', 'B', 1, 1, 1),
 ('D', 'C', 1, 2, 2),
 ('D', 'D', 0, 3, 0)]

El resultado obtenido es el siguiente:

![title](https://github.com/jscanass/modeling/blob/master/ej1_r.png?raw=true)

Entonces nuestras dos zonas quedan así: A, B con base en A y D,C con base en D. La elección de las bases es coherente con el peso de las zonas (las dos más altas) y la división de las zonas coincide con la sumatoria de los pesos por cada área generada. El viaje ponderado se calculó de la siguiente forma: ir de la base A a el destino B pesa 1 mas el viaje entre la base D al destino C que vale 2, para un viaje ponderado total de 3.

## Función de Optimización:

In [7]:
import pulp
import networkx
#Function to return results for p-median problem
#Ar - list of areas
#Di - distance dictionary, where Di[s][d] is the distance between source area a and destination d
#Co - contiguity dictionary, where Co[a] returns a list of areas adjacent to area a
#Ca - call dictionary, for Ca[a] returns the expected number of calls in area a
#Ta - total number of areas to partition set into
#In - exceptable level of inequality between loads, so passing 0.1 would mean top could have
# (Total Calls/Ta)*1.1 total calls in the partitioned area
#Th - threshold to consider source to destination as viable decision variables
def PMed(Ar,Di,Co,Ca,Ta,In,Th):
    #Setting max/min number of calls allowable in a created area
    SumCalls = sum(Ca.values())
    MaxIneq = (SumCalls/Ta)*(1 + In)
    MinIneq = (SumCalls/Ta)*(1 - In)
    #Turn contiguity Co into networkx graph
    G = networkx.Graph()
    for i in Ar:
        for j in Co[i]:
            G.add_edge(i,j)
    #Creating threshold vectors for decision variables
    NearAreas = {}
    Thresh = []
    for s in Ar:
        NearAreas[s] = []
        for d in Ar:
            if Di[s][d] < Th:
                Thresh.append((s,d))
                NearAreas[s].append(d)
    #Set up the problem
    P = pulp.LpProblem("P-Median",pulp.LpMinimize)

    #Decision variables
    assign_areas = pulp.LpVariable.dicts("Sources and Destinations", [(s,d) for (s,d) in Thresh], lowBound=0, upBound=1, cat=pulp.LpInteger)
    y_vars = pulp.LpVariable.dicts("SourceAreas", [s for s in Ar], lowBound=0, upBound=1, cat=pulp.LpInteger)
    #Function to minimize
    P += pulp.lpSum(Ca[d]*Di[s][d]*assign_areas[(s,d)] for (s,d) in Thresh)
    #Constraint Max number of beats
    P += pulp.lpSum(y_vars[s] for s in Ar) == Ta
    #Constraint No offbeat if local beat is not assigned first line
    #Second part is contiguity constraint
    for (s,d) in Thresh:
        P += assign_areas[(s,d)] - y_vars[s] <= 0
        #Only do this if source and destination not the same
        if s != d:
            #Identifying locations contiguous in nearest path
            both = set(networkx.shortest_path(G,s,d)) & set(Co[d])
            #Or if nearer to source
            nearer = [a for a in Co[d] if Di[s][a] < Di[s][d]]
            comb = list( both | set(nearer) ) #Combining all nearer areas, should always have at least one
            #Contiguity constraint
            P += pulp.lpSum(assign_areas[(s,a)] for a in comb if a in NearAreas[s]) >= assign_areas[(s,d)]
    #Constraint - every destination area must be covered at least once
    #Second line is not too high of workload, third is not too low
    for (sl,dl) in zip(Ar,Ar):
        P += pulp.lpSum(assign_areas[(s,dl)] for s in NearAreas[dl]) == 1
        P += pulp.lpSum(assign_areas[(sl,d)]*Ca[d] for d in NearAreas[sl]) <= MaxIneq
        P += pulp.lpSum(assign_areas[(sl,d)]*Ca[d] for d in NearAreas[sl]) >= MinIneq*y_vars[sl]
    #solve the model, note if you do not have CPLEX change this to whatever solver you want
    P.solve()
    stat = pulp.LpStatus[P.status]
    if stat != "Optimal":
        print("Status is %s" % (stat))
        return stat
    else:
        print("Status is %s, The total weighted travel is %d" % (stat,pulp.value(P.objective)))
        results = []
        for (s,d) in Thresh:
            if assign_areas[(s,d)].varValue == 1.0:
                results.append((s,d,Di[s][d],Ca[d],Ca[d]*Di[s][d]))
                
        return results

## Otros ejemplos:

TODO: Revisar cada uno

In [8]:
Areas = ['A','B','C','D']
Calls = {'A': 4,
'B': 1,
'C': 2,
'D': 3}
Distances = {'A': {'A': 0, 'B': 1, 'C': 2, 'D': 3},
'B': {'A': 1, 'B': 0, 'C': 1, 'D': 2},
'C': {'A': 2, 'B': 1, 'C': 0, 'D': 1},
'D': {'A': 3, 'B': 2, 'C': 1, 'D': 0}}
Contiguity = {'A': ['B'],
'B': ['A','C'],
'C': ['B','D'],
'D': ['C']}
PMed(Ar=Areas,Di=Distances,Co=Contiguity,Ca=Calls,Ta=2,In=0.2,Th=4)

Status is Optimal, The total weighted travel is 3


[('A', 'A', 0, 4, 0),
 ('A', 'B', 1, 1, 1),
 ('D', 'C', 1, 2, 2),
 ('D', 'D', 0, 3, 0)]

In [9]:
Areas = ['A','B','C','C2','D']
Calls = {'A': 2,
'B': 1,
'C': 1,
'C2':1,
'D': 8}
Distances = {'A': {'A': 0, 'B': 1, 'C': 2,'C2':3, 'D': 4},
            'B': {'A': 1, 'B': 0, 'C': 1, 'C2':2, 'D': 3},
            'C': {'A': 2, 'B': 1, 'C': 0, 'C2': 1, 'D':2},
            'C2':{'A': 3, 'B': 2, 'C': 1, 'C2': 0, 'D':1},
            'D': {'A': 4, 'B': 3, 'C': 2, 'C2':1, 'D': 0}}
Contiguity = {'A': ['B'],
'B': ['A','C','C2'],
'C': ['A','B','C2'],
'C2': ['B','C2','D'],
'D': ['C2']}
PMed(Ar=Areas,Di=Distances,Co=Contiguity,Ca=Calls,Ta=2,In=0.2,Th=4)

Status is Infeasible


'Infeasible'

In [10]:
PMed(Ar=Areas,Di=Distances,Co=Contiguity,Ca=Calls,Ta=1,In=0.2,Th=4)

Status is Optimal, The total weighted travel is 17


[('C2', 'A', 3, 2, 6),
 ('C2', 'B', 2, 1, 2),
 ('C2', 'C', 1, 1, 1),
 ('C2', 'C2', 0, 1, 0),
 ('C2', 'D', 1, 8, 8)]

In [11]:
#4 areas con los pesos anteriores
PMed(Ar=Areas,Di=Distances,Co=Contiguity,Ca=Calls,Ta=4,In=0.2,Th=4)

Status is Infeasible


'Infeasible'

In [12]:
#pesos iguales
Areas = ['A','B','C','D']
Calls = {'A': 1,
'B': 1,
'C': 1,
'D': 1}
Distances = {'A': {'A': 0, 'B': 1, 'C': 2, 'D': 3},
                'B': {'A': 1, 'B': 0, 'C': 1, 'D': 2},
                'C': {'A': 2, 'B': 1, 'C': 0, 'D': 1},
                'D': {'A': 3, 'B': 2, 'C': 1, 'D': 0}}
Contiguity = {'A': ['B'],
'B': ['A','C'],
'C': ['B','C'],
'D': ['C']}
PMed(Ar=Areas,Di=Distances,Co=Contiguity,Ca=Calls,Ta=2,In=0.2,Th=4)

Status is Optimal, The total weighted travel is 2


[('B', 'A', 1, 1, 1),
 ('B', 'B', 0, 1, 0),
 ('C', 'C', 0, 1, 0),
 ('C', 'D', 1, 1, 1)]

In [13]:
#pesos iguales con 4 areas 
Areas = ['A','B','C','D']
Calls = {'A': 1,
'B': 1,
'C': 1,
'D': 1}
Distances = {'A': {'A': 0, 'B': 1, 'C': 2, 'D': 3},
                'B': {'A': 1, 'B': 0, 'C': 1, 'D': 2},
                'C': {'A': 2, 'B': 1, 'C': 0, 'D': 1},
                'D': {'A': 3, 'B': 2, 'C': 1, 'D': 0}}
Contiguity = {'A': ['B'],
'B': ['A','C'],
'C': ['B','C'],
'D': ['C']}
PMed(Ar=Areas,Di=Distances,Co=Contiguity,Ca=Calls,Ta=4,In=0.2,Th=4)

Status is Optimal, The total weighted travel is 0


[('A', 'A', 0, 1, 0),
 ('B', 'B', 0, 1, 0),
 ('C', 'C', 0, 1, 0),
 ('D', 'D', 0, 1, 0)]

In [14]:
#help(pulp.LpProblem())

In [15]:
Areas = ['A','B','C']
Calls = {'A': 1,
'B': 1,
'C': 1,}
Distances = {'A': {'A': 0, 'B': 1, 'C': 2},
                'B': {'A': 1, 'B': 0, 'C': 1,},
                'C': {'A': 2, 'B': 1, 'C': 0, },
                'D': {'A': 3, 'B': 2, 'C': 1,}}
Contiguity = {'A': ['B'],
'B': ['A','C'],
'C': ['B'],
             }
PMed(Ar=Areas,Di=Distances,Co=Contiguity,Ca=Calls,Ta=3,In=0.2,Th=4)

Status is Optimal, The total weighted travel is 0


[('A', 'A', 0, 1, 0), ('B', 'B', 0, 1, 0), ('C', 'C', 0, 1, 0)]

In [16]:
Areas = ['A','B','C']
Calls = {'A': 3,
'B': 1,
'C': 1,}
Distances = {'A': {'A': 0, 'B': 1, 'C': 2},
                'B': {'A': 1, 'B': 0, 'C': 1,},
                'C': {'A': 2, 'B': 1, 'C': 0, },
                'D': {'A': 3, 'B': 2, 'C': 1,}}
Contiguity = {'A': ['B'],
'B': ['A','C'],
'C': ['B'],
             }
PMed(Ar=Areas,Di=Distances,Co=Contiguity,Ca=Calls,Ta=2,In=0.2,Th=4)

Status is Optimal, The total weighted travel is 1


[('A', 'A', 0, 3, 0), ('C', 'B', 1, 1, 1), ('C', 'C', 0, 1, 0)]

## Ejemplos asociados a la matriz de Intensidad:

En el problema real se parte de una matriz de intensdidad que contiene las predicciones. En el primer ejemplo se supone que nuestra predicción coincide con la matriz identidad de 4x4

In [17]:
intensity_matrix = np.identity(4)
intensity_matrix

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [18]:
# Nombramos las áreas con su componente 'ij'
Areas = []
for i in range(0, np.shape(intensity_matrix)[0]):
    for j in range(0, np.shape(intensity_matrix)[1]):
        Areas.append(str(i)+str(j))
Areas

['00',
 '01',
 '02',
 '03',
 '10',
 '11',
 '12',
 '13',
 '20',
 '21',
 '22',
 '23',
 '30',
 '31',
 '32',
 '33']

In [19]:
# Cada área tiene un valor de predicción 
# dada en la matrix de intensidad
Calls = {}
pred_values = np.matrix.flatten(intensity_matrix)
for index in range(0,np.shape(Areas)[0]):
    Calls.update({Areas[index]: pred_values[index]})
Calls

{'00': 1.0,
 '01': 0.0,
 '02': 0.0,
 '03': 0.0,
 '10': 0.0,
 '11': 1.0,
 '12': 0.0,
 '13': 0.0,
 '20': 0.0,
 '21': 0.0,
 '22': 1.0,
 '23': 0.0,
 '30': 0.0,
 '31': 0.0,
 '32': 0.0,
 '33': 1.0}

In [20]:
# El diccionario de distancias calcula la distancia
# de cada área a las demás
Distances = {}
unit = 1
row, column = np.shape(intensity_matrix)
coords = []
for n in range(0,row):
    for m in range(0, column):
        coords.append((n*unit,m*unit))
distance_matrix = distance.cdist(coords, coords, 'cityblock')
for i in range(0, np.shape(Areas)[0]):
    dictionary = dict(zip(Areas, distance_matrix[i]))
    Distances.update({Areas[i]:dictionary})
Distances

{'00': {'00': 0.0,
  '01': 1.0,
  '02': 2.0,
  '03': 3.0,
  '10': 1.0,
  '11': 2.0,
  '12': 3.0,
  '13': 4.0,
  '20': 2.0,
  '21': 3.0,
  '22': 4.0,
  '23': 5.0,
  '30': 3.0,
  '31': 4.0,
  '32': 5.0,
  '33': 6.0},
 '01': {'00': 1.0,
  '01': 0.0,
  '02': 1.0,
  '03': 2.0,
  '10': 2.0,
  '11': 1.0,
  '12': 2.0,
  '13': 3.0,
  '20': 3.0,
  '21': 2.0,
  '22': 3.0,
  '23': 4.0,
  '30': 4.0,
  '31': 3.0,
  '32': 4.0,
  '33': 5.0},
 '02': {'00': 2.0,
  '01': 1.0,
  '02': 0.0,
  '03': 1.0,
  '10': 3.0,
  '11': 2.0,
  '12': 1.0,
  '13': 2.0,
  '20': 4.0,
  '21': 3.0,
  '22': 2.0,
  '23': 3.0,
  '30': 5.0,
  '31': 4.0,
  '32': 3.0,
  '33': 4.0},
 '03': {'00': 3.0,
  '01': 2.0,
  '02': 1.0,
  '03': 0.0,
  '10': 4.0,
  '11': 3.0,
  '12': 2.0,
  '13': 1.0,
  '20': 5.0,
  '21': 4.0,
  '22': 3.0,
  '23': 2.0,
  '30': 6.0,
  '31': 5.0,
  '32': 4.0,
  '33': 3.0},
 '10': {'00': 1.0,
  '01': 2.0,
  '02': 3.0,
  '03': 4.0,
  '10': 0.0,
  '11': 1.0,
  '12': 2.0,
  '13': 3.0,
  '20': 1.0,
  '21': 2.0,
  '2

In [21]:
# Contiguidad
min_distance = 1 
Contiguity = {}
areas3 = list(Distances.keys())
distances3 = list(Distances.values())
for i in range(0, np.shape(areas3)[0]):  
    cond = {k:v for (k,v) in distances3[i].items() if v==min_distance}
    cond = list(cond.keys())
    Contiguity.update({areas3[i]:cond})
Contiguity

{'00': ['01', '10'],
 '01': ['00', '02', '11'],
 '02': ['01', '03', '12'],
 '03': ['02', '13'],
 '10': ['00', '11', '20'],
 '11': ['01', '10', '12', '21'],
 '12': ['02', '11', '13', '22'],
 '13': ['03', '12', '23'],
 '20': ['10', '21', '30'],
 '21': ['11', '20', '22', '31'],
 '22': ['12', '21', '23', '32'],
 '23': ['13', '22', '33'],
 '30': ['20', '31'],
 '31': ['21', '30', '32'],
 '32': ['22', '31', '33'],
 '33': ['23', '32']}

In [22]:
PMed(Ar=Areas,Di=Distances,Co=Contiguity,Ca=Calls,Ta=2,In=0.2,Th=4)

# Interpretación 
# Resultado para cada tupla:
# 1: Origen
# 2: Destino
# 3: Distancia Origen-Destino
# 4: Peso del Destino
# 5: Viaje Origen-Destino

# El viaje ponderado total se calcula 
# sumando la quinta componente de cada tupla


Status is Optimal, The total weighted travel is 4


[('11', '00', 2.0, 1.0, 2.0),
 ('11', '01', 1.0, 0.0, 0.0),
 ('11', '02', 2.0, 0.0, 0.0),
 ('11', '03', 3.0, 0.0, 0.0),
 ('11', '11', 0.0, 1.0, 0.0),
 ('11', '12', 1.0, 0.0, 0.0),
 ('22', '10', 3.0, 0.0, 0.0),
 ('22', '13', 2.0, 0.0, 0.0),
 ('22', '20', 2.0, 0.0, 0.0),
 ('22', '21', 1.0, 0.0, 0.0),
 ('22', '22', 0.0, 1.0, 0.0),
 ('22', '23', 1.0, 0.0, 0.0),
 ('22', '30', 3.0, 0.0, 0.0),
 ('22', '31', 2.0, 0.0, 0.0),
 ('22', '32', 1.0, 0.0, 0.0),
 ('22', '33', 2.0, 1.0, 2.0)]

El resultado lo podemos ver con la siguiente imagen: 
    
![title](https://github.com/jscanass/modeling/blob/master/ej_mi_.png?raw=true)

En este caso partimos de predicciones de crímenes en la diagonal y después de optimizar se divide la zona en dos, donde los colores oscuros de cada tonalidad son las bases. En nuestro caso el viaje ponderado total es 4, este es aportado por el viaje de 22 a 33 y de 11 a 00, 2 cada uno, como se ve en la 5ta componente del resultado. 

Este resultado es esperado pues los únicos valores que aportan al viaje ponderado total son los de la diagonal por lo que las bases efectivamente deben estar ahí y los viajes se hacen entre diagonales pero pasando por otras áreas, esto debido a la distancia utilizada: [distancia Manhattan](https://es.wikipedia.org/wiki/Geometr%C3%ADa_del_taxista).


In [23]:
# Ahora si queremos dividir la zona en 4 obtendremos las
# bases en las diagonales y el viaje ponderado total 0.
# La forma de las áreas generadas no interesa porque no 
# aportan a la predicción, estas solo dependen de los 
# parámetros de la predicción

PMed(Ar=Areas,Di=Distances,Co=Contiguity,Ca=Calls,Ta=4,In=0.2,Th=4)

Status is Optimal, The total weighted travel is 0


[('00', '00', 0.0, 1.0, 0.0),
 ('11', '01', 1.0, 0.0, 0.0),
 ('11', '02', 2.0, 0.0, 0.0),
 ('11', '03', 3.0, 0.0, 0.0),
 ('11', '11', 0.0, 1.0, 0.0),
 ('11', '12', 1.0, 0.0, 0.0),
 ('22', '10', 3.0, 0.0, 0.0),
 ('22', '13', 2.0, 0.0, 0.0),
 ('22', '20', 2.0, 0.0, 0.0),
 ('22', '21', 1.0, 0.0, 0.0),
 ('22', '22', 0.0, 1.0, 0.0),
 ('22', '23', 1.0, 0.0, 0.0),
 ('22', '30', 3.0, 0.0, 0.0),
 ('22', '31', 2.0, 0.0, 0.0),
 ('33', '32', 1.0, 0.0, 0.0),
 ('33', '33', 0.0, 1.0, 0.0)]

In [24]:
# Función para convertir la matriz de intensidad a
# los parametros de la función de intensidad

def Opt_Parameters(intensity_matrix,unit=1):
    
    Areas = []
    for i in range(0, np.shape(intensity_matrix)[0]):
        for j in range(0, np.shape(intensity_matrix)[1]):
            Areas.append(str(i)+str(j))
    
    Calls = {}
    pred_values = np.matrix.flatten(intensity_matrix)
    for index in range(0,np.shape(Areas)[0]):
        Calls.update({Areas[index]: pred_values[index]})
    
    Distances = {}
    row, column = np.shape(intensity_matrix)
    coords = []
    for n in range(0,row):
        for m in range(0, column):
            coords.append((n*unit,m*unit))
    distance_matrix = distance.cdist(coords, coords, 'cityblock')
    for i in range(0, np.shape(Areas)[0]):
        dictionary = dict(zip(Areas, distance_matrix[i]))
        Distances.update({Areas[i]:dictionary})
        
    Contiguity = {}
    areas3 = list(Distances.keys())
    distances3 = list(Distances.values())
    for i in range(0, np.shape(areas3)[0]):  
        cond = {k:v for (k,v) in distances3[i].items() if v==unit}
        cond = list(cond.keys())
        Contiguity.update({areas3[i]:cond})
    
    return Areas, Calls, Distances, Contiguity  

Ahora vamos a generar los siguientes ejemplos: 

![title](https://raw.githubusercontent.com/jscanass/modeling/master/ej2.png)

In [25]:
# Ejemplo 1: Even Calls

intensity_matrix1 = np.full((6,6),1)
intensity_matrix1

array([[1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1]])

In [26]:
Areas, Calls, Distances, Contiguity = Opt_Parameters(intensity_matrix1)
PMed(Ar=Areas,Di=Distances,Co=Contiguity,Ca=Calls,Ta=2,In=0.2,Th=10)

#In - exceptable level of inequality between loads, so passing 0.1 would mean top could have
# (Total Calls/Ta)*1.1 total calls in the partitioned area
#Th - threshold to consider source to destination as viable decision variables

Status is Optimal, The total weighted travel is 78


[('13', '00', 4.0, 1, 4.0),
 ('13', '01', 3.0, 1, 3.0),
 ('13', '02', 2.0, 1, 2.0),
 ('13', '03', 1.0, 1, 1.0),
 ('13', '04', 2.0, 1, 2.0),
 ('13', '05', 3.0, 1, 3.0),
 ('13', '10', 3.0, 1, 3.0),
 ('13', '11', 2.0, 1, 2.0),
 ('13', '12', 1.0, 1, 1.0),
 ('13', '13', 0.0, 1, 0.0),
 ('13', '14', 1.0, 1, 1.0),
 ('13', '15', 2.0, 1, 2.0),
 ('13', '20', 4.0, 1, 4.0),
 ('13', '22', 2.0, 1, 2.0),
 ('13', '23', 1.0, 1, 1.0),
 ('13', '24', 2.0, 1, 2.0),
 ('13', '25', 3.0, 1, 3.0),
 ('13', '33', 2.0, 1, 2.0),
 ('13', '34', 3.0, 1, 3.0),
 ('42', '21', 3.0, 1, 3.0),
 ('42', '30', 3.0, 1, 3.0),
 ('42', '31', 2.0, 1, 2.0),
 ('42', '32', 1.0, 1, 1.0),
 ('42', '35', 4.0, 1, 4.0),
 ('42', '40', 2.0, 1, 2.0),
 ('42', '41', 1.0, 1, 1.0),
 ('42', '42', 0.0, 1, 0.0),
 ('42', '43', 1.0, 1, 1.0),
 ('42', '44', 2.0, 1, 2.0),
 ('42', '45', 3.0, 1, 3.0),
 ('42', '50', 3.0, 1, 3.0),
 ('42', '51', 2.0, 1, 2.0),
 ('42', '52', 1.0, 1, 1.0),
 ('42', '53', 2.0, 1, 2.0),
 ('42', '54', 3.0, 1, 3.0),
 ('42', '55', 4.0, 1

In [27]:
# Ejemplo 2: 1 Hot Spot

intensity_matrix2 = np.full((6,6),1)
intensity_matrix2[0,0] = 10
intensity_matrix2[0,1] = 5
intensity_matrix2[1,0] = 5
intensity_matrix2

array([[10,  5,  1,  1,  1,  1],
       [ 5,  1,  1,  1,  1,  1],
       [ 1,  1,  1,  1,  1,  1],
       [ 1,  1,  1,  1,  1,  1],
       [ 1,  1,  1,  1,  1,  1],
       [ 1,  1,  1,  1,  1,  1]])

In [28]:
Areas, Calls, Distances, Contiguity = Opt_Parameters(intensity_matrix2)
PMed(Ar=Areas,Di=Distances,Co=Contiguity,Ca=Calls,Ta=2,In=0.2,Th=10)

Status is Optimal, The total weighted travel is 96


[('00', '00', 0.0, 10, 0.0),
 ('00', '01', 1.0, 5, 5.0),
 ('00', '02', 2.0, 1, 2.0),
 ('00', '03', 3.0, 1, 3.0),
 ('00', '10', 1.0, 5, 5.0),
 ('00', '11', 2.0, 1, 2.0),
 ('00', '12', 3.0, 1, 3.0),
 ('00', '20', 2.0, 1, 2.0),
 ('00', '21', 3.0, 1, 3.0),
 ('00', '30', 3.0, 1, 3.0),
 ('00', '40', 4.0, 1, 4.0),
 ('00', '50', 5.0, 1, 5.0),
 ('34', '04', 3.0, 1, 3.0),
 ('34', '05', 4.0, 1, 4.0),
 ('34', '13', 3.0, 1, 3.0),
 ('34', '14', 2.0, 1, 2.0),
 ('34', '15', 3.0, 1, 3.0),
 ('34', '22', 3.0, 1, 3.0),
 ('34', '23', 2.0, 1, 2.0),
 ('34', '24', 1.0, 1, 1.0),
 ('34', '25', 2.0, 1, 2.0),
 ('34', '31', 3.0, 1, 3.0),
 ('34', '32', 2.0, 1, 2.0),
 ('34', '33', 1.0, 1, 1.0),
 ('34', '34', 0.0, 1, 0.0),
 ('34', '35', 1.0, 1, 1.0),
 ('34', '41', 4.0, 1, 4.0),
 ('34', '42', 3.0, 1, 3.0),
 ('34', '43', 2.0, 1, 2.0),
 ('34', '44', 1.0, 1, 1.0),
 ('34', '45', 2.0, 1, 2.0),
 ('34', '51', 5.0, 1, 5.0),
 ('34', '52', 4.0, 1, 4.0),
 ('34', '53', 3.0, 1, 3.0),
 ('34', '54', 2.0, 1, 2.0),
 ('34', '55', 3.0, 

In [29]:
# Ejemplo 3: 1 Hot spot Near Middle

intensity_matrix3 = np.full((6,6),1)
intensity_matrix3[1,1] = 5
intensity_matrix3[1,2] = 5
intensity_matrix3[1,3] = 5
intensity_matrix3[2,1] = 5
intensity_matrix3[2,2] = 10
intensity_matrix3[2,3] = 5
intensity_matrix3[3,1] = 5
intensity_matrix3[3,2] = 5
intensity_matrix3[3,3] = 5

intensity_matrix3

array([[ 1,  1,  1,  1,  1,  1],
       [ 1,  5,  5,  5,  1,  1],
       [ 1,  5, 10,  5,  1,  1],
       [ 1,  5,  5,  5,  1,  1],
       [ 1,  1,  1,  1,  1,  1],
       [ 1,  1,  1,  1,  1,  1]])

In [30]:
Areas, Calls, Distances, Contiguity = Opt_Parameters(intensity_matrix3)
PMed(Ar=Areas,Di=Distances,Co=Contiguity,Ca=Calls,Ta=2,In=0.2,Th=10)

Status is Optimal, The total weighted travel is 122


[('21', '00', 3.0, 1, 3.0),
 ('21', '01', 2.0, 1, 2.0),
 ('21', '02', 3.0, 1, 3.0),
 ('21', '10', 2.0, 1, 2.0),
 ('21', '11', 1.0, 5, 5.0),
 ('21', '12', 2.0, 5, 10.0),
 ('21', '20', 1.0, 1, 1.0),
 ('21', '21', 0.0, 5, 0.0),
 ('21', '22', 1.0, 10, 10.0),
 ('21', '30', 2.0, 1, 2.0),
 ('21', '31', 1.0, 5, 5.0),
 ('21', '40', 3.0, 1, 3.0),
 ('21', '41', 2.0, 1, 2.0),
 ('21', '50', 4.0, 1, 4.0),
 ('21', '51', 3.0, 1, 3.0),
 ('33', '03', 3.0, 1, 3.0),
 ('33', '04', 4.0, 1, 4.0),
 ('33', '05', 5.0, 1, 5.0),
 ('33', '13', 2.0, 5, 10.0),
 ('33', '14', 3.0, 1, 3.0),
 ('33', '15', 4.0, 1, 4.0),
 ('33', '23', 1.0, 5, 5.0),
 ('33', '24', 2.0, 1, 2.0),
 ('33', '25', 3.0, 1, 3.0),
 ('33', '32', 1.0, 5, 5.0),
 ('33', '33', 0.0, 5, 0.0),
 ('33', '34', 1.0, 1, 1.0),
 ('33', '35', 2.0, 1, 2.0),
 ('33', '42', 2.0, 1, 2.0),
 ('33', '43', 1.0, 1, 1.0),
 ('33', '44', 2.0, 1, 2.0),
 ('33', '45', 3.0, 1, 3.0),
 ('33', '52', 3.0, 1, 3.0),
 ('33', '53', 2.0, 1, 2.0),
 ('33', '54', 3.0, 1, 3.0),
 ('33', '55', 4.

In [31]:
# Ejemplo 4: 2 Hot Spot

intensity_matrix4 = np.full((6,6),1)
intensity_matrix4[0,0] = 10
intensity_matrix4[0,1] = 5
intensity_matrix4[1,0] = 5
intensity_matrix4[5,5] = 10
intensity_matrix4[4,5] = 5
intensity_matrix4[5,4] = 5
intensity_matrix4

array([[10,  5,  1,  1,  1,  1],
       [ 5,  1,  1,  1,  1,  1],
       [ 1,  1,  1,  1,  1,  1],
       [ 1,  1,  1,  1,  1,  1],
       [ 1,  1,  1,  1,  1,  5],
       [ 1,  1,  1,  1,  5, 10]])

In [32]:
Areas, Calls, Distances, Contiguity = Opt_Parameters(intensity_matrix4)
PMed(Ar=Areas,Di=Distances,Co=Contiguity,Ca=Calls,Ta=2,In=0.2,Th=10)

Status is Optimal, The total weighted travel is 125


[('10', '00', 1.0, 10, 10.0),
 ('10', '01', 2.0, 5, 10.0),
 ('10', '02', 3.0, 1, 3.0),
 ('10', '03', 4.0, 1, 4.0),
 ('10', '04', 5.0, 1, 5.0),
 ('10', '10', 0.0, 5, 0.0),
 ('10', '11', 1.0, 1, 1.0),
 ('10', '12', 2.0, 1, 2.0),
 ('10', '13', 3.0, 1, 3.0),
 ('10', '14', 4.0, 1, 4.0),
 ('10', '20', 1.0, 1, 1.0),
 ('10', '21', 2.0, 1, 2.0),
 ('10', '22', 3.0, 1, 3.0),
 ('10', '23', 4.0, 1, 4.0),
 ('10', '30', 2.0, 1, 2.0),
 ('10', '31', 3.0, 1, 3.0),
 ('10', '32', 4.0, 1, 4.0),
 ('10', '40', 3.0, 1, 3.0),
 ('10', '41', 4.0, 1, 4.0),
 ('10', '50', 4.0, 1, 4.0),
 ('55', '05', 5.0, 1, 5.0),
 ('55', '15', 4.0, 1, 4.0),
 ('55', '24', 4.0, 1, 4.0),
 ('55', '25', 3.0, 1, 3.0),
 ('55', '33', 4.0, 1, 4.0),
 ('55', '34', 3.0, 1, 3.0),
 ('55', '35', 2.0, 1, 2.0),
 ('55', '42', 4.0, 1, 4.0),
 ('55', '43', 3.0, 1, 3.0),
 ('55', '44', 2.0, 1, 2.0),
 ('55', '45', 1.0, 5, 5.0),
 ('55', '51', 4.0, 1, 4.0),
 ('55', '52', 3.0, 1, 3.0),
 ('55', '53', 2.0, 1, 2.0),
 ('55', '54', 1.0, 5, 5.0),
 ('55', '55', 0.0

## Ejemplos con las predicciones reales:

(TODO)